In [1]:
import os

from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from keras.optimizers import SGD, RMSprop

from data_generator import WrapperImageDataGenerator

Using TensorFlow backend.


In [2]:
TRAIN_DATA_DIR = '/home/vs/Source/visionhack/data/trainset/data_to_fit_jpg_train/'
VALID_DATA_DIR = '/home/vs/Source/visionhack/data/trainset/data_to_fit_jpg_valid/'
IMAGE_SHAPE = (299, 299)
BATCH_SIZE = 64

train_data_generator = WrapperImageDataGenerator(featurewise_center=False,
                                                samplewise_center=False,
                                                featurewise_std_normalization=False,
                                                samplewise_std_normalization=False,
                                                zca_whitening=False,
                                                zca_epsilon=1e-6,
                                                rotation_range=10.,
                                                width_shift_range=10.,
                                                height_shift_range=10.,
                                                shear_range=0.,
                                                zoom_range=0.,
                                                channel_shift_range=0.,
                                                fill_mode='nearest',
                                                cval=0.,
                                                horizontal_flip=False,
                                                vertical_flip=False,
                                                rescale=None,
                                                data_format=K.image_data_format(),
                                                preprocessing_function=preprocess_input)

train_generator = train_data_generator.flow_from_directory(TRAIN_DATA_DIR,
                                                           target_size=IMAGE_SHAPE,
                                                           batch_size=BATCH_SIZE,
                                                           class_mode="categorical",
                                                           shuffle=True,
                                                           color_mode='rgb')


valid_data_generator = WrapperImageDataGenerator(rescale=None,
                                                 data_format=K.image_data_format(),
                                                 preprocessing_function=preprocess_input)

valid_generator = valid_data_generator.flow_from_directory(VALID_DATA_DIR,
                                                           target_size=IMAGE_SHAPE,
                                                           batch_size=BATCH_SIZE,
                                                           class_mode="categorical",
                                                           shuffle=False,
                                                           color_mode='rgb')

Found 44354 images belonging to 512 classes.
Found 10795 images belonging to 512 classes.


In [3]:
MODEL_DATA_DIR = 'data/models'
LR = 0.045

#callbacks
checkpoint_path = 'checkpoints_weights.{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint_path = os.path.join(MODEL_DATA_DIR, checkpoint_path)
checkpointer = ModelCheckpoint(filepath=checkpoint_path, verbose=1, period=1)

stoper = EarlyStopping(min_delta=0.001, patience=7)

reducer = ReduceLROnPlateau(factor=0.94, patience=2, min_lr=LR * 0.001, verbose=1)

log_path = 'log.csv'
log_path = os.path.join(MODEL_DATA_DIR, log_path)
logger = CSVLogger(filename=log_path, append=True)

callbacks = [checkpointer, stoper, reducer]

In [ ]:
# create the base pre-trained model
base_model = InceptionV3(input_shape=(IMAGE_SHAPE[0], IMAGE_SHAPE[1], 3), weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
#x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(9, activation='sigmoid')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 299, 299, 3)   0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 149, 149, 32)  864         input_1[0][0]                    
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, 149, 149, 32)  96          conv2d_1[0][0]                   
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 149, 149, 32)  0           batch_normalization_1[0][0]      
___________________________________________________________________________________________

In [ ]:
TRAIN_DATA_DIR = ''
EPOCHS = 100
IMAGES_ON_TRAIN = 44354
IMAGES_ON_VALIDATION = 10795

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False
    
optimizer = RMSprop(lr=LR, decay=0.9, epsilon=1)
    
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['MSE'])

model.fit_generator(train_generator, 
                    epochs=EPOCHS,
                    steps_per_epoch = int(IMAGES_ON_TRAIN / BATCH_SIZE),
                    verbose=1, 
                    validation_data=valid_generator,
                    callbacks=callbacks,  
                    validation_steps=int(0.1 * IMAGES_ON_VALIDATION / BATCH_SIZE), 
                    class_weight=None, 
                    initial_epoch=0)

Epoch 1/100
693/693 [==============================] - 839s - loss: 0.3335 - mean_squared_error: 0.0863 - val_loss: 0.1407 - val_mean_squared_error: 0.0189
Epoch 2/100
693/693 [==============================] - 815s - loss: 0.2876 - mean_squared_error: 0.0689 - val_loss: 0.1216 - val_mean_squared_error: 0.0148
Epoch 3/100
693/693 [==============================] - 825s - loss: 0.2761 - mean_squared_error: 0.0651 - val_loss: 0.1287 - val_mean_squared_error: 0.0165
Epoch 4/100
693/693 [==============================] - 823s - loss: 0.2691 - mean_squared_error: 0.0628 - val_loss: 0.1142 - val_mean_squared_error: 0.0131
Epoch 5/100
217/693 [========>.....................] - ETA: 540s - loss: 0.2670 - mean_squared_error: 0.0622

In [ ]:
FINETUNE_EPOCHS = 100
FINETUNE_LR = 0.0001

for i, layer in enumerate(base_model.layers):
    print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
    layer.trainable = False
for layer in model.layers[249:]:
    layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate

model.compile(optimizer=SGD(lr=FINETUNE_LR, momentum=0.9), loss='binary_crossentropy', metrics=['MSE'])

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit_generator(train_generator, 
                    epochs=FINETUNE_EPOCHS,
                    steps_per_epoch = int(IMAGES_ON_EPOCHS / BATCH_SIZE)
                    verbose=1, 
                    validation_data=valid_generator,
                    callbacks=callbacks,  
                    validation_steps=int(0.1 * IMAGES_ON_EPOCHS / BATCH_SIZE), 
                    class_weight=None, 
                    initial_epoch=EPOCHS)